In [1]:
import random
import itertools
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
df = pd.read_csv('sp_500.csv')
tickers = df['Symbol'].tolist()[:100]
tickers = [ticker for ticker in tickers if ticker not in ['BRK.B', 'CTLT', 'BF.B', 'ABC', 'ATVI']]

In [3]:
start_date = dt.datetime(2022, 1, 1)
end_date = dt.datetime(2025, 2, 24)

n_tickers_selected = 4
n_combinations = 200

sampled_combinations = random.sample(list(itertools.combinations(tickers, n_tickers_selected)), n_combinations)

In [4]:
all_portfolios = []
rf = 0.0415

for combination in sampled_combinations:
    data = yf.download(list(combination), start= start_date, end = end_date, progress= False)["Close"]
    returns = data.pct_change().dropna()
    covariance_matrix = returns.cov()

    annual_returns = returns.mean() * 252
    annual_cov = covariance_matrix * 252

    # Numero de simulaciones de ponderaciones
    num_portafolios = 10000

    weights_array = np.zeros((num_portafolios, n_tickers_selected))
    returns_array = np.zeros(num_portafolios)
    risk_array = np.zeros(num_portafolios)
    sharpe_array = np.zeros(num_portafolios)

    for i in range(num_portafolios):
        # generas los pesos aleatorios para cada portafolio
        weights = np.random.random(n_tickers_selected)
        weights = weights / np.sum(weights)

        # obteniendo rendimiento y volatilidad esperada anual por portafolio
        portfolio_return = np.dot(annual_returns, weights)
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights)))

        # guardando historicos
        weights_array[i, :] = weights
        returns_array[i] = portfolio_return
        risk_array[i] = portfolio_risk
        sharpe_array[i] = (portfolio_return - rf) / portfolio_risk

    portfolios = pd.DataFrame({
        "Returns": returns_array,
        "Risk": risk_array,
        "Sharpe": sharpe_array
    })

    for i, symbol in enumerate(combination):
        portfolios[symbol + " Weight"] = weights_array[:, i]

    all_portfolios.append(portfolios)

YF.download() has changed argument auto_adjust default to True


In [5]:
todo = pd.concat(all_portfolios)

sorted_todo = todo.sort_values("Sharpe", ascending=False)

selected_rows = []
used_tickers = set()
for idx, row in sorted_todo.iterrows():
    # extract tickers from non-NaN weight columns
    row_tickers = [col.replace(" Weight", "") for col in todo.columns if col.endswith("Weight") and pd.notna(row[col])]
    if set(row_tickers) & used_tickers:
        continue
    selected_rows.append(row)
    used_tickers.update(row_tickers)
    if len(selected_rows) >= 10:
        break

top_10 = pd.DataFrame(selected_rows)
# Rename weight columns to only show the ticker name
top_10.rename(columns=lambda col: col.replace(" Weight", "") if col.endswith(" Weight") else col, inplace=True)

In [6]:
sharpe_df = []
for i in range(10):
    clean_todo = top_10.iloc[i:i+1]
    clean_todo = clean_todo.dropna(axis=1, how='all')
    sharpe_df.append(clean_todo)

In [7]:
def get_data(stocks, start_date, end_date):
    prices = yf.download(stocks, start_date, end_date, progress = False)["Close"]
    returns = prices.pct_change().dropna()
    mean_returns = returns.mean()
    cov_returns = returns.cov()
    return mean_returns, cov_returns, prices

In [8]:
def simular_precios(mean_returns, cov_returns, prices, num_dias, stocks, weights):

    N = 10000
    portfolio_return = np.dot(weights, mean_returns)

    portfolio_simulated_returns = np.zeros((num_dias, N)) # filas x columnas
    mean_returns = np.full(shape = (num_dias, len(stocks)), fill_value=portfolio_return)

    for m in range(N):
        L = np.linalg.cholesky(cov_returns)
        Z = np.random.normal(size=(num_dias, len(stocks))) # Matriz de n_dias (filas) y n_stocks (columnas)
        daily_returns = mean_returns.T + np.dot(L, Z.T)
        portfolio_simulated_returns [:, m] = np.cumprod(np.dot(weights, daily_returns) + 1)
    
    return portfolio_simulated_returns

In [9]:
max_sharpe_list = [sharpe_df[i] for i in range(10)]

In [10]:
start_date = dt.datetime(2022, 1, 1)
end_date = dt.datetime(2025, 2, 24)
stocks = max_sharpe_list[0].columns[3:-1].tolist()
weights = max_sharpe_list[0].values[0][3:-1]
num_dias = 14

In [11]:
mean_returns, cov_returns, prices = get_data(stocks, start_date, end_date)
portafolios_simulados = simular_precios(mean_returns, cov_returns, prices, num_dias, stocks, weights)

In [12]:
var_list = []

for i in range(len(max_sharpe_list)):
    start_date = dt.datetime(2022, 1, 1)
    end_date = dt.datetime(2025, 2, 24)
    stocks = max_sharpe_list[i].columns[3:-1].tolist()
    weights = max_sharpe_list[i].values[0][3:-1]

    mean_returns, cov_returns, prices = get_data(stocks, start_date, end_date)
    portafolios_simulados = simular_precios(mean_returns, cov_returns, prices, num_dias, stocks, weights)
    var = np.percentile(portafolios_simulados[-1, :], 5) - 1
    var_list.append(var)


In [13]:
for i in range(len(max_sharpe_list)):
    max_sharpe_list[i]['VaR'] = var_list[i]

In [14]:
for i in range(10):
    print(f"Portafolio {i+1}")
    display(max_sharpe_list[i])

Portafolio 1


,Returns,Risk,Sharpe,ALL,APH,BRO,CAH,VaR
3999,0.292198,0.189352,1.323981,0.046542,0.181244,0.079947,0.692266,-0.023616


Portafolio 2


,Returns,Risk,Sharpe,BSX,ACGL,BIIB,AIZ,VaR
4220,0.288098,0.187721,1.313642,0.665604,0.303111,0.005372,0.025913,-0.075199


Portafolio 3


...

,Returns,Risk,Sharpe,WRB,AMD,AMP,BK,VaR
2885,0.218994,0.208964,0.8494,0.108753,0.000793,0.350704,0.53975,-0.037689


Portafolio 9


,Returns,Risk,Sharpe,CHRW,ALLE,AXP,AMGN,VaR
3034,0.194167,0.204104,0.747984,0.016851,0.010912,0.431748,0.540489,-0.050205


Portafolio 10


,Returns,Risk,Sharpe,AMT,ANSS,CBOE,CBRE,VaR
8911,0.17878,0.199564,0.687898,0.002738,0.036004,0.882914,0.078344,-0.064684
